# 課題
* 市区町村名の重複を解決する。
* GitHubへのPush方法を調査する。

In [5]:
import xml.etree.ElementTree as ET 
import requests
import csv

proxies = {
  'http': 'gienah.in.dcs.co.jp:8080'
}
url = 'http://www.data.jma.go.jp/developer/xml/feed/extra.xml'

response = requests.get(url, proxies=proxies)

with open('alert.csv', 'w', encoding='utf_8', newline='') as csvfile:

    writer = csv.writer(csvfile)
    writer.writerow(['種別','名称','エリア(市区町村)'])
    
    if response.status_code != 200:
        sys.exit()

    ns_feed = '{http://www.w3.org/2005/Atom}'
    ns_detail = '{http://xml.kishou.go.jp/jmaxml1/body/meteorology1/}'
    
    response.encoding = response.apparent_encoding
    root = ET.fromstring(response.text)
    
    for entry in root.iter(ns_feed+'entry'):
        
        #「気象警報・注意報（Ｈ２７）」を使用
        if entry.find(ns_feed+'title').text != '気象警報・注意報（Ｈ２７）':
            continue

        # 明細XMLを取得
        link = entry.find(ns_feed+'link').attrib['href']        
        print(link)
        
        response_detail = requests.get(link, proxies=proxies)
        response_detail.encoding = response_detail.apparent_encoding        
        detail_root = ET.fromstring(response_detail.text)
        
        # 「気象警報・注意報（市町村等）」を使用
        warning = detail_root.find(ns_detail + 'Body/' + ns_detail +'Warning[@type="気象警報・注意報（市町村等）"]')
        
        if warning is None:
            continue
                    
        for item in warning.iter(ns_detail + 'Item'):
                        

            name = item.find(ns_detail + 'Kind/' + ns_detail + 'Name')
            status = item.find(ns_detail + 'Kind/' + ns_detail + 'Status')
            area = item.find(ns_detail + 'Area/' + ns_detail + 'Name')

            if kind_code is None or kind_name is None:
                continue
            if status.text == '解除': #「解除」ステータスは除く
                continue
                
            if '警報' in (name.text):
                kind = '警報'
            elif '注意報' in (name.text):
                kind = '注意報'

            #print([level_id, level, kind_code.text, kind_name.text, area_name.text, status.text])
            writer.writerow([kind, name.text, area.text])

http://www.data.jma.go.jp/developer/xml/data/298e4de0-665f-372d-a594-d6a0aeaa3b3a.xml
http://www.data.jma.go.jp/developer/xml/data/cda9ded8-3692-3ae8-aa06-7c344eb1af01.xml
http://www.data.jma.go.jp/developer/xml/data/3e91126b-eb47-3461-94bb-b9b927e9634d.xml
http://www.data.jma.go.jp/developer/xml/data/1c09be2d-c4a8-3e5f-afba-5ae463a953b2.xml
http://www.data.jma.go.jp/developer/xml/data/1d13ae0c-3e48-35b3-abc0-567c171186c7.xml
http://www.data.jma.go.jp/developer/xml/data/0b75ac75-031f-3e5c-a749-f0b43c3fdcf4.xml
http://www.data.jma.go.jp/developer/xml/data/63114e15-49a9-3996-9d20-b1bec27e0da2.xml
http://www.data.jma.go.jp/developer/xml/data/076b9adc-de69-3d1b-b750-c7f39cc80b29.xml
http://www.data.jma.go.jp/developer/xml/data/3eaedb8d-200a-3db5-8579-cb0e5a4ca1a0.xml


In [5]:
!pip --proxy=gienah.in.dcs.co.jp:8080 --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org install requests

You should consider upgrading via the 'c:\users\u031330\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
